In [ ]:
pip install matplotlib

In [ ]:
pip install seaborn

In [ ]:
import psycopg2
import pandas as pd
import psycopg2.extras
import seaborn as sns
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
class PostgresConnection(object):
    def __init__(self):
        self.connection = psycopg2.connect(database="ecomdb",
                                           user = "postgres",
                                           password = "ahanaf12",
                                           host = "127.0.0.1",
                                           port = "5432")

    def getConnection(self):
        print("successfully connected to database")
        return self.connection

In [ ]:
con = PostgresConnection().getConnection()

# Q1: Find the division wise total_sale_price joining fact table and respective dimension table

In [ ]:
def changetype(x):
    x['sales']=x['sales'].astype('float64')  

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT s.division, SUM(t.total_price) " \
              "FROM star_schema.\"Fact_table\" t " \
              "JOIN star_schema.\"Store_dim\" s on s.store_key=t.store_key " \
                "GROUP BY CUBE(s.division) " \
                "ORDER BY s.division "
cur.execute(select_stmt)
records = cur.fetchall()

df = pd.DataFrame(records, columns=['division', 'sales'])
df =df.drop(df.index[-1])
df = df.set_index(['division'])

changetype(df)
df.plot.pie(y='sales')
plt.figure(figsize=(10,5))
df.plot.bar(y='sales')

division_total_sales = df
division_total_sales

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT s.year, SUM(t.total_price) " \
              "FROM star_schema.\"Fact_table\" t " \
              "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                "GROUP BY CUBE(s.year) " \
                "ORDER BY s.year "
cur.execute(select_stmt)
records = cur.fetchall()

df = pd.DataFrame(records, columns=['year', 'sales'])
df = df.drop(df.index[-1])
df = df.set_index(['year'])

changetype(df)
df.plot.pie(y='sales')
plt.figure(figsize=(10,5))
df.plot.bar(y='sales')

year_total_sales = df
year_total_sales

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT s.district, SUM(t.total_price) " \
              "FROM star_schema.\"Fact_table\" t " \
              "JOIN star_schema.\"Store_dim\" s on s.store_key=t.store_key " \
                "GROUP BY CUBE(s.district) " \
                "ORDER BY s.district "
cur.execute(select_stmt)
records = cur.fetchall()

df = pd.DataFrame(records, columns=['district', 'sales'])
df = df.drop(df.index[-1])
df = df.set_index(['district'])

changetype(df)
df.plot.pie(y='sales')
plt.figure(figsize=(10,5))
df.plot.bar(y='sales')

district_total_sales = df
district_total_sales

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT s.month, SUM(t.total_price) " \
              "FROM star_schema.\"Fact_table\" t " \
              "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                "GROUP BY CUBE(s.month) " \
                "ORDER BY s.month "
cur.execute(select_stmt)
records = cur.fetchall()

df = pd.DataFrame(records, columns=['month', 'sales'])
df = df.drop(df.index[-1])
df = df.set_index(['month'])

changetype(df)
df.plot.pie(y='sales')
plt.figure(figsize=(10,5))
df.plot.bar(y='sales')

month_total_sales = df
month_total_sales

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT s.name, SUM(t.total_price) " \
              "FROM star_schema.\"Fact_table\" t " \
              "JOIN star_schema.\"Customer_dim\" s on s.customer_key=t.customer_key " \
                "GROUP BY CUBE(s.name) " \
                "ORDER BY s.name "
cur.execute(select_stmt)
records = cur.fetchall()

df = pd.DataFrame(records, columns=['Customer Name', 'sales'])
df = df.drop(df.index[-1])
df = df.set_index(['Customer Name'])

customer_total_sales = df
customer_total_sales

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT s.bank_name, SUM(t.total_price) " \
              "FROM star_schema.\"Fact_table\" t " \
              "JOIN star_schema.\"Trans_dim\" s on s.payment_key=t.payment_key " \
                "GROUP BY CUBE(s.bank_name) " \
                "ORDER BY s.bank_name "
cur.execute(select_stmt)
records = cur.fetchall()

df = pd.DataFrame(records, columns=['Bank Name', 'sales'])
df = df.drop(df.index[-1])
df = df.set_index(['Bank Name'])

changetype(df)
df.plot.pie(y='sales')
plt.legend(loc='upper right')
plt.figure(figsize=(10,5))
df.plot.bar(y='sales')


bank_total_sales = df
bank_total_sales

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT s.trans_type, SUM(t.total_price) " \
              "FROM star_schema.\"Fact_table\" t " \
              "JOIN star_schema.\"Trans_dim\" s on s.payment_key=t.payment_key " \
                "GROUP BY CUBE(s.trans_type) " \
                "ORDER BY s.trans_type "
cur.execute(select_stmt)
records = cur.fetchall()

df = pd.DataFrame(records, columns=['Transaction Type', 'sales'])
df = df.drop(df.index[-1])
df = df.set_index(['Transaction Type'])

changetype(df)
df.plot.pie(y='sales')
plt.figure(figsize=(10,5))
df.plot.bar(y='sales')

transaction_total_sales = df
transaction_total_sales

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT s.division, SUM(t.total_price) " \
              "FROM star_schema.\"Fact_table\" t " \
              "JOIN star_schema.\"Store_dim\" s on s.store_key=t.store_key " \
                "WHERE s.division = 'BARISAL'" \
                "GROUP BY s.division"
cur.execute(select_stmt)
records = cur.fetchall()

df = pd.DataFrame(records, columns=['District', 'Total Sales'])
df = df.set_index(['District'])

barisal_total_sales = df
barisal_total_sales

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT s.year, SUM(t.total_price) " \
              "FROM star_schema.\"Fact_table\" t " \
              "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                "WHERE s.year = 2015 " \
                "GROUP BY s.year " 
cur.execute(select_stmt)
records = cur.fetchall()

df = pd.DataFrame(records, columns=['Year', 'Total Sales'])
df = df.set_index(['Year'])

year_2015_total_sales = df
year_2015_total_sales

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT s.year, p.division, SUM(t.total_price) " \
              "FROM star_schema.\"Fact_table\" t " \
              "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                "JOIN star_schema.\"Store_dim\" p on p.store_key=t.store_key " \
                "WHERE s.year = 2015 AND p.division = 'BARISAL'" \
                "GROUP BY s.year, p.division" 
cur.execute(select_stmt)
records = cur.fetchall()

df = pd.DataFrame(records, columns=['Year', 'Division', 'Total Sales'])
df = df.set_index(['Division'])

year_2015_barisal_total_sales = df
year_2015_barisal_total_sales

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT store_key, item_name, quantity " \
              "FROM (" \
                    "SELECT s.store_key, p.item_name, SUM(t.quantity) AS quantity, " \
                    "ROW_NUMBER() OVER (PARTITION BY s.store_key ORDER BY SUM(t.quantity) DESC) AS row_num " \
                    "FROM star_schema.\"Fact_table\" t " \
                    "JOIN star_schema.\"Store_dim\" s on s.store_key=t.store_key " \
                    "JOIN star_schema.\"Item_dim\" p on p.item_key=t.item_key " \
                    "GROUP BY s.store_key, p.item_name " \
              ") subquery " \
              "WHERE row_num <= 3 " \
              "ORDER BY store_key, quantity DESC"  
cur.execute(select_stmt)
records = cur.fetchall()

df = pd.DataFrame(records, columns=['Store Key', 'Item Name', 'Quantity Sales'])

store_item_quantity_sales = df

print(store_item_quantity_sales.columns)

for index, row in df.iterrows():
    if index % 3 == 0:
        print("Store Key: {}".format(row['Store Key']))
    print("\t\tItem Name: {}, Quantity Sold: {}".format(row['Item Name'], row['Quantity Sales']))

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()

select_stmt = "SELECT item_name, quarter " \
              "FROM (" \
                    "SELECT p.item_name, s.quarter, SUM(t.total_price) AS sum, " \
                    "RANK() OVER (PARTITION BY p.item_name ORDER BY SUM(t.total_price) ASC) AS rank " \
                    "FROM star_schema.\"Fact_table\" t " \
                    "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                    "JOIN star_schema.\"Item_dim\" p on p.item_key=t.item_key " \
                    "GROUP BY p.item_name, s.quarter" \
              ") subquery " \
              "WHERE rank = 1 " \
              "ORDER BY item_name"
cur.execute(select_stmt)
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Item Name", "Worst Quarter"])

items_worst_quarter = df
items_worst_quarter

#for index, row in df.iterrows():
#    print("{} -> Worst Quarter: {}".format(row['Item Name'], row['Worst Quarter']))

In [ ]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

days = 1000 # input("Days: ") 

select_stmt = "SELECT DISTINCT p.item_name " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                "JOIN star_schema.\"Item_dim\" p on p.item_key=t.item_key " \
                "JOIN star_schema.\"Trans_dim\" r on r.payment_key=t.payment_key " \
                "WHERE s.date > (CURRENT_DATE - INTERVAL '{} day') AND " \
                "(r.trans_type = 'card' OR r.trans_type = 'mobile')".format(days) 
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Item Name"]) 
df = df.set_index(['Item Name'])

items_since_days = df 
items_since_days

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT s.item_name, p.division, SUM(t.total_price) " \
              "FROM star_schema.\"Fact_table\" t " \
              "JOIN star_schema.\"Item_dim\" s on s.item_key=t.item_key " \
                "JOIN star_schema.\"Store_dim\" p on p.store_key=t.store_key " \
                "GROUP BY s.item_name, p.division " \
                "ORDER BY s.item_name, SUM(t.total_price) DESC"
cur.execute(select_stmt)
records = cur.fetchall()

df = pd.DataFrame(records, columns=['Item Name', 'Division', 'Total Sales'])

item_division_sales = df
item_division_sales

for index, row in df.iterrows():
    if index % 7 == 0:
        print("Item Name: {}".format(row['Item Name']))
    print("\t\tDivision: {}, Total Sales: {}".format(row['Division'], row['Total Sales']))

In [ ]:
con = PostgresConnection().getConnection()
cur = con.cursor()
select_stmt = "SELECT s.store_key, p.month, AVG(t.total_price) " \
              "FROM star_schema.\"Fact_table\" t " \
              "JOIN star_schema.\"Store_dim\" s on s.store_key=t.store_key " \
                "JOIN star_schema.\"Time_dim\" p on p.time_key=t.time_key " \
                "GROUP BY s.store_key, p.month " \
                "ORDER BY s.store_key, p.month"
cur.execute(select_stmt)
records = cur.fetchall()

df = pd.DataFrame(records, columns=['Store Key', 'Month', 'Average Sales'])

store_monthly_sales = df
store_monthly_sales

for index, row in df.iterrows():
    if index % 12 == 0:
        print("Store Key: {}".format(row['Store Key']))
    print("\t\tMonth: {}, Average Sales: {}".format(row['Month'], row['Average Sales']))

# Decision 1a: Find top 10 stores with the least sales during the given input duration

In [ ]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

year = 2018 # input("Year: ") 
month = 5 # input("Month: ") 
day_start = 1 # input("Starting Date: ") 
day_end = 30 # input("End date: ") 

select_stmt = "SELECT store_key, sales " \
              "FROM (" \
                "SELECT p.store_key, SUM(t.total_price) AS sales, " \
                "RANK() OVER (ORDER BY SUM(t.total_price) ASC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                "JOIN star_schema.\"Store_dim\" p on p.store_key=t.store_key " \
                "WHERE s.year = {} AND s.month = {} AND s.day >= {} AND s.day < {} " \
                "GROUP BY p.store_key " \
               ") subquery " \
               "WHERE rank <= 10 " \
               "ORDER BY rank".format(year, month, day_start, day_end) 
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Store Key", "Sales"]) 
df = df.set_index(['Store Key'])

stores_least_sales = df 
stores_least_sales

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

# Decision 1b: Find top 3 districts under each division which were the most porfitable during input duration

In [ ]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

year = 2018 # input("Year: ") 
month = 5 # input("Month: ") 
day_start = 1 # input("Starting Date: ") 
day_end = 30 # input("End date: ") 

select_stmt = "SELECT division, district, sales " \
              "FROM (" \
                "SELECT p.division, p.district, SUM(t.total_price) AS sales, " \
                "RANK() OVER (PARTITION BY p.division ORDER BY SUM(t.total_price) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                "JOIN star_schema.\"Store_dim\" p on p.store_key=t.store_key " \
                "WHERE s.year = {} AND s.month = {} AND s.day >= {} AND s.day < {} AND " \
                "p.division != p.district " \
                "GROUP BY p.division, p.district " \
               ") subquery " \
               "WHERE rank <= 3 " \
               "ORDER BY division".format(year, month, day_start, day_end) 
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Division", "District", "Sales"]) 
df = df.set_index(['Division'])

most_profitable_districts = df 
most_profitable_districts

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

# Decision 2a: Find top 30 customers who made the highest purchases between 2 pm - 5 pm in 2020

In [ ]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT name, sales " \
              "FROM (" \
                "SELECT p.name, SUM(t.total_price) AS sales, " \
                "RANK() OVER (ORDER BY SUM(t.total_price) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                "JOIN star_schema.\"Customer_dim\" p on p.customer_key=t.customer_key " \
                "WHERE s.hour >= 14 AND s.hour <= 17 AND s.year = 2020 " \
                "GROUP BY p.name " \
               ") subquery " \
               "WHERE rank <= 30 " \
               "ORDER BY rank".format(year, month, day_start, day_end) 
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Name", "Sales"]) 
df = df.set_index(['Name'])

idle_period_customers = df 
idle_period_customers

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

# Decision 2b: Find top 20 customers who made purchases on highest number of days during a year

In [ ]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT name, days " \
              "FROM (" \
                "SELECT p.name, COUNT(DISTINCT s.date) AS days, " \
                "RANK() OVER (ORDER BY COUNT(DISTINCT s.date) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                "JOIN star_schema.\"Customer_dim\" p on p.customer_key=t.customer_key " \
                "WHERE s.year = 2016 " \
                "GROUP BY p.name " \
               ") subquery " \
               "WHERE rank <= 20 " \
               "ORDER BY rank".format(year, month, day_start, day_end) 
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Name", "Sales"]) 
df = df.set_index(['Name'])

persistent_customers = df 
persistent_customers

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

# Decision 3a: Most popular Soda Beverage

In [75]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT item_name, sales " \
              "FROM (" \
                "SELECT p.item_name, SUM(t.total_price) AS sales, " \
                "RANK() OVER (ORDER BY SUM(t.total_price) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
              "JOIN star_schema.\"Item_dim\" p on p.item_key=t.item_key " \
                "WHERE p.description = 'a. Beverage - Soda' " \
                "GROUP BY p.item_name " \
               ") subquery " \
              "ORDER BY rank"
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Name", "Sales"]) 
df = df.set_index(['Name'])

popular_pepsi = df 
popular_pepsi

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

successfully connected to database


,Sales
Name,
Pepsi - 12 oz cans,534982.25
Diet Coke - 12 oz cans,523575.50
Coke Classic 12 oz cans,522173.75
Sprite - 12 oz cans,519480.50
Coke Zero Sugar 12 oz cans,367380.00
Dr. Pepper - 12 oz cans,363935.00
Diet Dr. Pepper - 12 oz cans,361887.50
Mountain Dew - 12 oz cans,359433.75
A&W Root Beer - 12 oz cans,271664.5


# Decision 3b: Most popular product of each type(description) and the supplier

In [87]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT description, item_name, supplier_final " \
              "FROM (" \
                "SELECT p.description, p.item_name, SUM(t.total_price) AS sales, " \
                "CONCAT  (supplier, ', ', man_country) AS supplier_final, " \
                "RANK() OVER (PARTITION BY p.description ORDER BY SUM(t.total_price) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
              "JOIN star_schema.\"Item_dim\" p on p.item_key=t.item_key " \
                "GROUP BY p.description, p.item_name, supplier_final " \
               ") subquery " \
              "WHERE rank = 1 ORDER BY rank"
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Item Type", "Item Name", "Supplier"]) 
df = df.set_index(['Item Type'])

supplier_popular = df 
supplier_popular

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

successfully connected to database


,Item Name,Supplier
Item Type,,
a. Beverage - Soda,Pepsi - 12 oz cans,"BIGSO AB, Lithuania"
a. Beverage Sparkling Water,Topo Chico 12 oz glass,"MAESA SAS, United States"
Beverage - Energy/Protein,Red Bull 12oz,"MAESA SAS, United States"
Beverage - Energy/Protein,Monster Java 15 oz,"Friedola 1888 GmbH, Germany"
Beverage - Gatorade,Gatorade Variety 20 oz,"DENIMACH LTD, Bangladesh"
Beverage - Iced Tea,Tejava Unsweetened Black Tea 16.9oz,"CHERRY GROUP CO.,LTD, China"
Beverage - Juice,Tropicana Apple Juice 10 oz 100%,"Friedola 1888 GmbH, Germany"
Beverage Water,Sobe Zero Lifewater Variety 20 oz,"NINGBO SEDUNO IMP & EXP CO.LTD, Cambodia"
Coffee Cream,Half & Half -Land o Lakes Mini Moo's,"BIGSO AB, Lithuania"


# Decision 4a: Top 10 stores with the highest quantity of products sold

In [88]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT store_key, quantity_sold " \
              "FROM (" \
                "SELECT p.store_key, SUM(t.quantity) AS quantity_sold, " \
                "RANK() OVER (ORDER BY SUM(t.quantity) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                "JOIN star_schema.\"Store_dim\" p on p.store_key=t.store_key " \
                "GROUP BY p.store_key " \
               ") subquery " \
               "WHERE rank <= 10 " \
               "ORDER BY rank"
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Store Key", "Quantity Sold"]) 
df = df.set_index(['Store Key'])

quantity_sales = df 
quantity_sales

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

successfully connected to database


,Quantity Sold
Store Key,
S00328,8980
S0010,8965
S0039,8909
S00601,8904
S00462,8895
S00357,8888
S0080,8886
S00627,8877
S00360,8865


# Decision 4b: Top 5 stores with the highest sales of products in Carton Unit

In [91]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT store_key, cartons_sold " \
              "FROM (" \
                "SELECT p.store_key, SUM(t.quantity) AS cartons_sold, " \
                "RANK() OVER (ORDER BY SUM(t.quantity) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
                "JOIN star_schema.\"Store_dim\" p on p.store_key=t.store_key " \
                "WHERE t.unit = 'cartons' " \
                "GROUP BY p.store_key " \
               ") subquery " \
               "WHERE rank <= 5 " \
               "ORDER BY rank"
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Store Key", "Quantity Sold"]) 
df = df.set_index(['Store Key'])

cartons_sales = df 
cartons_sales

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

successfully connected to database


,Quantity Sold
Store Key,
S00269,98
S00395,95
S00159,89
S00111,79
S00193,79


# Decision 5a: Maximum and Minimum Medicine Sales(quantity) during each quarter per year

In [113]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT item_name, year, quarter, quantity_sold " \
              "FROM (" \
                "SELECT p.item_name, s.year, s.quarter, SUM(t.quantity) AS quantity_sold, " \
                "RANK() OVER (PARTITION BY s.year, p.item_name ORDER BY SUM(t.quantity) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
              "JOIN star_schema.\"Item_dim\" p on p.item_key=t.item_key " \
                "WHERE p.description = 'Medicine' " \
                "GROUP BY p.item_name, s.year, s.quarter " \
               ") subquery " \
               "WHERE rank = 1 OR rank = 4 " \
               "ORDER BY year, item_name, quantity_sold DESC"
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Name", "Year", "Quarter", "Quantity"]) 
df = df.set_index(['Name'])

medicine_quantity = df 
medicine_quantity

#for index, row in df.iterrows():
#    print("{}".format(row['Item Name']))

successfully connected to database


,Year,Quarter,Quantity
Name,,,
Advil 2 pill packets,2014,Q2,849
Advil 2 pill packets,2014,Q1,639
Tylenol Extra Strength 2 pill packets,2014,Q4,894
Tylenol Extra Strength 2 pill packets,2014,Q1,673
Advil 2 pill packets,2015,Q4,911
Advil 2 pill packets,2015,Q2,829
Tylenol Extra Strength 2 pill packets,2015,Q1,850
Tylenol Extra Strength 2 pill packets,2015,Q4,799
Advil 2 pill packets,2016,Q1,819


# Decision 5b: Top 2 most in-demand food per type(description) during each quarter of the year 2020

In [123]:
con = PostgresConnection().getConnection() 
cur = con.cursor() 

select_stmt = "SELECT description, quarter, item_name, quantity_sold " \
              "FROM (" \
                "SELECT p.description, p.item_name, s.quarter, SUM(t.quantity) AS quantity_sold, " \
                "RANK() OVER (PARTITION BY p.description, s.quarter ORDER BY SUM(t.quantity) DESC) AS rank " \
                "FROM star_schema.\"Fact_table\" t " \
                "JOIN star_schema.\"Time_dim\" s on s.time_key=t.time_key " \
              "JOIN star_schema.\"Item_dim\" p on p.item_key=t.item_key " \
                "WHERE p.description like 'Food%' AND s.year = 2020 " \
                "GROUP BY p.description, s.quarter, p.item_name " \
               ") subquery " \
               "WHERE rank <= 2 " \
               "ORDER BY description, quarter, rank"
cur.execute(select_stmt) 
records = cur.fetchall()

df = pd.DataFrame(records, columns=["Type", "Quarter", "Item Name", "Quantity Sold"]) 

popular_food = df 
popular_food

for index, row in df.iterrows():
    if index % 8 == 0:
        print("{}".format(row['Type']))
        print(("\t{}").format(row['Quarter']))
    elif index % 2 == 0:
        print(("\t{}").format(row['Quarter']))
    print(("\t\t{}").format(row['Item Name']))

successfully connected to database
Food - Chips
	Q1
		Frito Classic Mix Variety
		Chex Mix Original
	Q2
		Pringles Snak Stacks Variety
		Gardettos Snacks
	Q3
		Cheetos Flamin' Hot 1 oz
		Pringles Snak Stacks Variety
	Q4
		Cheetos Flamin' Hot 1 oz
		Frito Fiesta Flavors Variety
Food - Chocolate
	Q1
		Mars Chocolate Favorites
		Reese's Peanutbutter Cups 2/pk
	Q2
		Reese's Peanutbutter Cups 2/pk
		Mars Mini Mix
	Q3
		M&M Variety Full Size Bags
		Reese's Pieces Peanutbutter Candy
	Q4
		Hershey Factory Favorites Mix
		Reese's Pieces Peanutbutter Candy
Food - Healthy
	Q1
		Nature Valley Biscuit Sandwich Variety Pack 
		Go-Gurt Low Fat Yogurt, Strwbry & Berry 
	Q2
		Sargento String Cheese 0.75oz
		Belvita Hard Biscuits Cinnamon
	Q3
		Chewy Granola Variety Bars Quaker
		Kind  Bars Variety Pack 1.4 oz 
	Q4
		Nat.Valley PeanutButter Protein Bars
		Quaker Instant Oatmeal Maple Brn Sugar
Food - Nuts
	Q1
		Deluxe Mixed Nuts with Seasalt
		Blue Diamond Almonds Rstd, Sltd
	Q2
		Planters Nut-trition 1